# 네이버 데이터랩 쇼핑인사이트 인기검색어 크롤링

네이버 데이터랩 API가 있지만 API로는 쇼핑인사이트의 인기검색어를 가져올 수 없다.  
따라서 파이썬의 셀레니움을 이용하여 데이터랩의 인기검색어를 가져와봤다.

 

인기검색어가 20개만 노출되어 있기 때문에 다음으로 넘기는 버튼을 클릭하기 위해서는 bs4보다는 selenium이 더 적합하다.

In [1]:
import time
from selenium import webdriver

driver = webdriver.Chrome('./chromedriver')

# 쇼핑인사이트 이동
path = 'https://datalab.naver.com/shoppingInsight/sCategory.naver'
driver.get(path)

# 기기별 전체 선택
driver.find_element_by_xpath('//*[@id="18_device_0"]').click()
# 성별 전체 선택
driver.find_element_by_xpath('//*[@id="19_gender_0"]').click()
# 연령별 전체 선택
driver.find_element_by_xpath('//*[@id="20_age_0"]').click()


# 분류 & 기간 선택
time.sleep(20)


# 조회하기 클릭
driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div[1]/div/a').click()
time.sleep(3)


keyword_list = []

for p in range(0, 25):
    # 인기검색어 가져오기
    for i in range(1, 21):
        keyword_path = f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[{i}]/a'
        keyword_list.append(driver.find_element_by_xpath(keyword_path).text)

    # 다음 페이지 넘기기 
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/a[2]').click()
    time.sleep(1)



# 종료하기

분류와 검색기간의 조건은 설정하지 않고 기기별/성별/연령은 전체선택하도록 코드를 작성해보았다.   
최근 한 달 간 패션의류 카테고리의 인기검색어 500개를 가져오게 된다.


키워드를 받아올 때는 따로 시간지연(time.sleep)은 입력해두지 않았는데 필요에 따라 추가하면 된다.   

   
   
중간에 조건설정을 위해 time.sleep(20)을 넣어준 것은 좀 야매이긴하다,,,   
분류와 날짜 조건 코드를 변경하는 것보다 손으로 바꾸는게 편할거같아서 20초의 공백시간을 입력해두었다.   
처음 크롬창이 나타났을 때 20초 동안 분류나 조건을 변경하고 조회하기를 누르면 된다.

### 인기키워드 데이터프레임으로 만들기

받아온 리스트를 데이터프레임으로 만들어주었다. '순위\n키워드'의 형태로 데이터가 들어있기에 \n를 기준으로 split해준 후 각각 열 이름을 지정해주었다.  

In [2]:
import pandas as pd

pd.DataFrame(list(map((lambda x : x.split('\n')), keyword_list)), columns = ['순위', '인기검색어'])

,순위,인기검색어
0,1,원피스
1,2,트위드자켓
2,3,트렌치코트
3,4,나이키바람막이
4,5,블라우스
...,...,...
495,496,올젠
496,497,파타고니아
497,498,윈드브레이커
498,499,폴로자켓
